# Car price Predictions

In [ ]:
import  numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
car_data = pd.read_csv("/kaggle/input/car-price-estimation/datasets_33080_43333_car data.csv")
car_data

## EDA Steps

In [ ]:
car_data.info()

In [ ]:
car_data['Fuel_Type'].unique()

In [ ]:
car_data['Seller_Type'].unique()

In [ ]:
car_data['Transmission'].unique()

In [ ]:
car_data['Owner'].unique()

In [ ]:
car_data['Fuel_Type'].value_counts()

In [ ]:
car_data['Seller_Type'].value_counts()

In [ ]:
car_data['Transmission'].value_counts()

In [ ]:
car_data['Owner'].value_counts()

In [ ]:
car_data.isnull().sum()

In [ ]:
car_data.describe()

In [ ]:
car_data.corr()

In [ ]:
sns.heatmap(car_data.corr(), annot = True)

In [ ]:
car_data['Present Year'] = 2021

In [ ]:
car_data

In [ ]:
car_data['Number of Year'] = car_data['Present Year'] - car_data['Year']

In [ ]:
car_data

In [ ]:
car_data = car_data.drop(['Car_Name', 'Year','Present Year'], axis = 1)
car_data

In [ ]:
sns.pairplot(car_data)

In [ ]:
car_data = pd.get_dummies(car_data,drop_first = True)
car_data

In [ ]:
x = car_data.drop('Selling_Price', axis = 1)
y = car_data.Selling_Price

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size = 0.8,random_state = 42)

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
x_test.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_x = StandardScaler()
x_train = sc_x.fit_transform(x_train)
x_test = sc_x.transform(x_test)
sc_y = StandardScaler()


## Logistic Regression Model

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
LR_model = LinearRegression()
LR_model.fit(x_train, y_train)

In [ ]:
LR_model.score(x_train,y_train)

In [ ]:
LR_model.coef_

In [ ]:
LR_model.intercept_

In [ ]:
lr_pred = LR_model.predict(x_test)
lr_pred

## Ordinary Least Squares regression (OLS)

In [ ]:
import statsmodels.api as sm

In [ ]:
regressor_OLS = sm.OLS(endog=y, exog=x).fit()
regressor_OLS.summary()

In [ ]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test,lr_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test,lr_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test,lr_pred)))

## Random Forest Model Buliding

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf_reg = RandomForestRegressor()
rf_reg.fit(x_train, y_train)

In [ ]:
rf_reg.score(x_train,y_train)

In [ ]:
rf_reg.score(x_test, y_test)

In [ ]:
rf_pred = rf_reg.predict(x_test)

In [ ]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test,rf_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test,rf_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test,rf_pred)))

## Hyperparameter Tunning

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
 #Randomized Search CV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]


In [ ]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

print(random_grid)

In [ ]:
rf = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,scoring='neg_mean_squared_error', 
                               n_iter = 10, cv = 5, verbose=2, random_state=42, n_jobs = 1)

In [ ]:
rf_random.fit(x_train,y_train)

In [ ]:
rf_random.best_params_

In [ ]:
rf_random.best_score_

In [ ]:
predictions=rf_random.predict(x_test)

In [ ]:
sns.distplot(y_test-predictions)

In [ ]:
plt.scatter(y_test,predictions)


### RMSE Score

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
import pickle
# open a file, where you ant to store the data
file = open('random_forest_regression_model.pkl', 'wb')

# dump information to that file
pickle.dump(rf_random, file)